<a href="https://colab.research.google.com/github/sparks-baird/self-driving-lab-demo/blob/main/notebooks/escience/2.8.1-ax_service_existing_data_saasbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See https://github.com/facebook/Ax/issues/743

In [1]:
%pip install ax-platform

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 KB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.3 MB/s eta 0:00:00


In [2]:
# %% imports
import numpy as np
import pandas as pd

from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models

from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

unique_components = ["filler_A", "filler_B", "resin_A", "resin_B", "resin_C"]

X_train = np.array([
       [0.4, 0.4, 0. , 0. , 0.2],
       [0.5, 0. , 0. , 0.5, 0. ],
       [0.5, 0.3, 0. , 0.2, 0. ],
       [0.5, 0. , 0. , 0.5, 0. ],
       [0. , 0.6, 0.4, 0. , 0. ],
       [0.6, 0. , 0.4, 0. , 0. ],
       [0. , 0.6, 0.2, 0.2, 0. ]])

X_train = pd.DataFrame(X_train, columns=unique_components)
X_train

,filler_A,filler_B,resin_A,resin_B,resin_C
0,0.4,0.4,0.0,0.0,0.2
1,0.5,0.0,0.0,0.5,0.0
2,0.5,0.3,0.0,0.2,0.0
3,0.5,0.0,0.0,0.5,0.0
4,0.0,0.6,0.4,0.0,0.0
5,0.6,0.0,0.4,0.0,0.0
6,0.0,0.6,0.2,0.2,0.0


In [3]:
np.random.seed(10)
n_train = X_train.shape[0]
y_train = 100 * np.random.rand(n_train)
y_train

array([77.13206433,  2.07519494, 63.36482349, 74.88038825, 49.85070123,
       22.47966455, 19.80628648])

In [4]:
# Ax-specific
parameters = [
    {"name": component, "type": "range", "bounds": [0.0, 1.0]}
    for component in unique_components[:-1]
]
parameters

[{'name': 'filler_A', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'filler_B', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_A', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_B', 'type': 'range', 'bounds': [0.0, 1.0]}]

In [5]:
separator = " + "
composition_constraint = separator.join(unique_components[:-1]) + " <= 1.0"
composition_constraint

'filler_A + filler_B + resin_A + resin_B <= 1.0'

In [12]:
from botorch.models import SaasFullyBayesianSingleTaskGP
from ax.models.torch.botorch_modular.surrogate import Surrogate

# skip the pseudo-random suggested points by specifying a custom generation strategy
gs = GenerationStrategy(
    steps=[
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=Models.BOTORCH_MODULAR,
            model_kwargs={
                "surrogate": Surrogate(SaasFullyBayesianSingleTaskGP),
            },
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=3,  # Parallelism limit for this step, often lower than for Sobol
            # More on parallelism vs. required samples in BayesOpt:
            # https://ax.dev/docs/bayesopt.html#tradeoff-between-parallelism-and-total-number-of-trials
        ),
    ]
)
# setup the experiment
ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="dummy",
    parameters=parameters,
    parameter_constraints=[
        composition_constraint,
    ],
    minimize=True,
)

[INFO 02-18 03:26:34] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-18 03:26:34] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_A. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 03:26:34] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 03:26:34] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter resin_A. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 03:26:3

In [13]:
# attach the training data
for i in range(n_train):
    ax_client.attach_trial(X_train.iloc[i, :-1].to_dict())
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])

[INFO 02-18 03:26:40] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.4, 'filler_B': 0.4, 'resin_A': 0.0, 'resin_B': 0.0} as trial 0.
[INFO 02-18 03:26:40] ax.service.ax_client: Completed trial 0 with data: {'objective': (77.132064, None)}.
[INFO 02-18 03:26:40] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.5, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 0.5} as trial 1.
[INFO 02-18 03:26:40] ax.service.ax_client: Completed trial 1 with data: {'objective': (2.075195, None)}.
[INFO 02-18 03:26:40] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.5, 'filler_B': 0.3, 'resin_A': 0.0, 'resin_B': 0.2} as trial 2.
[INFO 02-18 03:26:40] ax.service.ax_client: Completed trial 2 with data: {'objective': (63.364823, None)}.
[INFO 02-18 03:26:40] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.5, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 0.5} as trial 3.
[INFO 02-18 03:26:40] ax.service.ax_client: Completed tr

In [14]:
# produce a *single* next suggested experiment, be sure to only run this once
next_experiment, trial_index = ax_client.get_next_trial()
print("next suggested experiment: ", next_experiment)

/usr/local/lib/python3.8/dist-packages/botorch/models/fully_bayesian.py:124: RuntimeWarning: Received a linear algebra error while sampling with Pyro. Adding a jitter of 1e-08 to the covariance matrix and retrying.
  warnings.warn(
[INFO 02-18 03:27:11] ax.service.ax_client: Generated new trial 7 with parameters {'filler_A': 0.0, 'filler_B': 0.0, 'resin_A': 1.0, 'resin_B': 0.0}.


next suggested experiment:  {'filler_A': 0.0, 'filler_B': 0.0, 'resin_A': 1.0, 'resin_B': 3.805414108826177e-10}


In [15]:
# note that the model fit is poor because of the toy data and randomly generated objective values
# (i.e. this is what we would expect: a bad fit, because the "true" values are nonsense)
best_parameters, metrics = ax_client.get_best_parameters()
print(best_parameters, metrics)

[WARNING 02-18 03:27:41] ax.modelbridge.cross_validation: Metric objective was unable to be reliably fit.
[WARNING 02-18 03:27:41] ax.service.utils.best_point: Model fit is poor; falling back on raw data for best point.
[WARNING 02-18 03:27:41] ax.service.utils.best_point: Model fit is poor and data on objective metric objective is noisy; interpret best points results carefully.


{'filler_A': 0.5, 'filler_B': 0.0, 'resin_A': 0.0, 'resin_B': 0.5} ({'objective': 2.07519493594015}, {'objective': {'objective': nan}})
